# Install & Import MediaPipe, OpenCV, Numpy
OpenCV for webcam, MediaPipe for pose detection, Numpy for calculations, winsound for alarm, and threading to run sound without blocking. The print statements explain what MediaPipe Pose does.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import winsound
import threading

# Print info about MediaPipe and Pose detection
print("MediaPipe installed successfully")
print("Test MediaPipe Pose Detection")
print("MediaPipe Pose gives you 33 body landmarks like: Nose, Shoulders, Elbows, Hips, Knees,")
print("Each landmark has: x coordinate, y coordinate, visibility")
print("We are drawing them using: mp_draw.draw_landmarks()")


MediaPipe installed successfully
Test MediaPipe Pose Detection
MediaPipe Pose gives you 33 body landmarks like: Nose, Shoulders, Elbows, Hips, Knees,
Each landmark has: x coordinate, y coordinate, visibility
We are drawing them using: mp_draw.draw_landmarks()


# Initialize MediaPipe Pose
Set up MediaPipe Pose with minimum confidence thresholds. mp_draw is used to draw landmarks on the video frames.

In [12]:
mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    min_detection_confidence=0.6,
                    min_tracking_confidence=0.6)

# Define Alarm Function
This function plays beep.wav alarm without stopping the webcam feed, the SND_ASYNC makes it non-blocking.

In [16]:
slight_audio = r"C:\Users\Admin\Downloads\audio_file\beep_slight.wav"
bad_audio = r"C:\Users\Admin\Downloads\audio_file\beep_bad.wav"

def play_alarm(audio_file):
    winsound.PlaySound(audio_file, winsound.SND_FILENAME | winsound.SND_ASYNC)

# Angle smoothing

In [13]:
alpha = 0.5
smoothed_angle = None

# Alarm flags

In [14]:
slight_alarm_triggered = False
bad_alarm_triggered = False

# Define Angle Calculation Function
This function calculates the angle at point b using coordinates of three points. We use it to measure the shoulder-to-hip tilt for posture detection.

In [15]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
              np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle

# Webcam Loop – Capture, Detect, Angle, Posture, Alarm
This cell handles the entire webcam workflow: capture frames, detect pose, calculate angles, smooth angles, determine posture, trigger alarm, and display results in real-time. This ensures your webcam runs smoothly without breaking between cells.

In [17]:
# Webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame for MediaPipe
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Check visibility 
        left_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
        right_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
        if left_vis < 0.5 or right_vis < 0.5:
            cv2.putText(image, "Move into view", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Posture Detection", image)
            if cv2.waitKey(1) == 27:
                break
            continue

        # Get coordinates
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

        # Calculate angle
        left_angle = calculate_angle(left_ear, left_shoulder, left_hip)
        right_angle = calculate_angle(right_ear, right_shoulder, right_hip)
        angle = (left_angle + right_angle) / 2

        # Smooth angle
        if smoothed_angle is None:
            smoothed_angle = angle
        else:
            smoothed_angle = alpha*angle + (1-alpha)*smoothed_angle

        # Determine posture & alarms
        if smoothed_angle > 150:  # Good posture
            posture_text = "Good Posture"
            color = (0,255,0)
            slight_alarm_triggered = False
            bad_alarm_triggered = False

        elif smoothed_angle > 140:  # Slight slouch
            posture_text = "Slight Slouch"
            color = (0,255,255)
            if not slight_alarm_triggered:
                threading.Thread(target=play_alarm, args=(slight_audio,), daemon=True).start()
                slight_alarm_triggered = True
            bad_alarm_triggered = False  # reset bad alarm

        else:  # Bad posture
            posture_text = "Bad Posture"
            color = (0,0,255)
            if not bad_alarm_triggered:
                threading.Thread(target=play_alarm, args=(bad_audio,), daemon=True).start()
                bad_alarm_triggered = True
            slight_alarm_triggered = False  # reset slight alarm

        # Draw skeleton & text 
        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.putText(image, f"Angle: {int(smoothed_angle)}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)
        cv2.putText(image, posture_text, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    else:
        cv2.putText(image, "No person detected", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)

    cv2.imshow("Posture Detection", image)

    # Press ESC to exit
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()